In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import joblib

# 1. Cargar archivo Excel (ajusta el nombre si tu archivo se llama distinto)
# Si lo subes por el explorador de Colab, suele quedar en la carpeta /content
data_path = "Life_Expectancy_Data.xlsx"
df = pd.read_excel(data_path)

# 2. Limpieza básica de nombres de columnas
df.columns = [c.strip() for c in df.columns]

# 3. Detectar la columna de Life Expectancy de forma robusta
target_candidates = [c for c in df.columns
                     if "life" in c.lower() and "expect" in c.lower()]

if not target_candidates:
    raise ValueError("No se encontró una columna que parezca 'Life expectancy'")

target_col = target_candidates[0]   # usamos la primera coincidencia

# 4. Eliminar filas sin dato de esperanza de vida
df = df.dropna(subset=[target_col])

# 5. Opcional: eliminar columna 'Status' si existe (simplifica el modelo)
if "Status" in df.columns:
    df = df.drop(columns=["Status"])

# 6. Separar variables explicativas (X) y variable objetivo (y)
X = df.drop(columns=[target_col])
y = df[target_col]

# 7. One-Hot Encoding de Country (si existe)
if "Country" in X.columns:
    X = pd.get_dummies(X, columns=["Country"], drop_first=True)

# 8. Rellenar valores faltantes numéricos con la mediana de cada columna
numeric_cols = X.select_dtypes(include=[np.number]).columns
X[numeric_cols] = X[numeric_cols].fillna(X[numeric_cols].median())

# 9. División Train / Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# 10. Escalado
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# 11. Entrenar regresión lineal
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

# 12. Evaluación
y_pred = linreg.predict(X_test_scaled)

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("======================================")
print("     MODELO REGRESIÓN LINEAL")
print("======================================")
print(f"R²   : {r2:.4f}")
print(f"RMSE : {rmse:,.4f}")

# 13. Guardar artefactos para el backend
feature_cols = X.columns.tolist()

joblib.dump(linreg,        "life_linreg_model.pkl")
joblib.dump(scaler,        "life_scaler.pkl")
joblib.dump(feature_cols,  "life_features.pkl")

print("\nArchivos generados:")
print("- life_linreg_model.pkl")
print("- life_scaler.pkl")
print("- life_features.pkl")


     MODELO REGRESIÓN LINEAL
R²   : 0.9551
RMSE : 1.9703

Archivos generados:
- life_linreg_model.pkl
- life_scaler.pkl
- life_features.pkl
